<a href="https://colab.research.google.com/github/ali-rn/project-3/blob/main/nnunet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/neheller/kits23

fatal: destination path 'kits23' already exists and is not an empty directory.


In [3]:
%cd kits23
!pip3 install .
!kits23_download_data

/content/kits23
Processing /content/kits23
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/deepmind/surface-distance.git to /tmp/pip-install-yeb20vw5/surface-distance_cf06553f731a4455a1ecb19890bfca5a
  Running command git clone --filter=blob:none --quiet https://github.com/deepmind/surface-distance.git /tmp/pip-install-yeb20vw5/surface-distance_cf06553f731a4455a1ecb19890bfca5a
  Resolved https://github.com/deepmind/surface-distance.git to commit 1f805cea446805e559badc80dd27f34175e4173d
  Preparing metadata (setup.py) ... done
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Created wheel for kits23: filename=kits23-0.1.4-py3-none-any.whl size=26451 sha256=d3b6775644e1089bf0b7ddc68fb6768dd38105a2d5d774b3396b5311b7f58e7e
  Stored in directory: /tmp/pip-ephem-wheel-cache-obo7nq18/wheels/52/e0/d1/1867083a301ed0c69dbf3f9d06ff506067c63d626d9529f29f
Successfully built kits23
  Attempting unins


Found 426 cases to download

Dowloading case_00588...: 100% 426/426 [42:46<00:00,  6.02s/it]


In [6]:
import os
from glob import glob

example_case = "/content/kits23/dataset/case_00000"
print("Contents of case_00000:")
print(os.listdir(example_case))

Contents of case_00000:
['instances', 'segmentation.nii.gz']


In [29]:
import os
import json
import nibabel as nib
from glob import glob
from shutil import copyfile

kits23_dir = "/content/kits23/dataset"
nnunet_dir = "/content/nnUNet_raw/Dataset502_KiTS23"
imagesTr_dir = os.path.join(nnunet_dir, "imagesTr")
labelsTr_dir = os.path.join(nnunet_dir, "labelsTr")

os.makedirs(imagesTr_dir, exist_ok=True)
os.makedirs(labelsTr_dir, exist_ok=True)

case_dirs = sorted(glob(os.path.join(kits23_dir, "case_*")))
skipped = 0

for case_path in case_dirs:
    case_id = os.path.basename(case_path)

    instance_files = glob(os.path.join(case_path, "instances", "*.nii.gz"))
    if not instance_files:
        print(f"No CT image found in {case_id}, skipping.")
        skipped += 1
        continue

    image_path = instance_files[0]
    label_path = os.path.join(case_path, "segmentation.nii.gz")

    if not os.path.exists(label_path):
        print(f"No label file for {case_id}, skipping.")
        skipped += 1
        continue

    target_image = os.path.join(imagesTr_dir, f"{case_id}_0000.nii.gz")
    target_label = os.path.join(labelsTr_dir, f"{case_id}.nii.gz")

    copyfile(image_path, target_image)
    copyfile(label_path, target_label)

print(f"Converted {len(case_dirs) - skipped} cases to nnUNet format.")
print(f"Skipped {skipped} cases.")

fixed_dataset_json = {
    "channel_names": {
        "0": "CT"
    },
    "labels": {
        "0": "background",
        "1": "kidney",
        "2": "tumor",
        "3": "cyst"
    },
    "numTraining": 489,
    "file_ending": ".nii.gz",
    "training": [
        {
            "image": f"./imagesTr/case_{i:05d}_0000.nii.gz",
            "label": f"./labelsTr/case_{i:05d}.nii.gz"
        } for i in range(489)
    ],
    "test": []
}

with open("/content/nnUNet_raw/Dataset502_KiTS23/dataset.json", "w") as f:
    json.dump(fixed_dataset_json, f, indent=4)

print("Fixed dataset.json written.")

Converted 489 cases to nnUNet format.
Skipped 0 cases.
Fixed dataset.json written.


In [15]:
!git clone https://github.com/MIC-DKFZ/nnUNet.git

Cloning into 'nnUNet'...
remote: Enumerating objects: 13673, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 13673 (delta 1), reused 0 (delta 0), pack-reused 13669 (from 3)
Receiving objects: 100% (13673/13673), 8.05 MiB | 25.67 MiB/s, done.
Resolving deltas: 100% (10466/10466), done.


In [16]:
# Step 3: Install nnUNet
%cd nnUNet
!pip install -e .
%cd ..

/content/nnUNet
Obtaining file:///content/nnUNet
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 M

/content


In [30]:
import os
os.environ["nnUNet_raw"] = "/content/nnUNet_raw"
os.environ["nnUNet_preprocessed"] = "/content/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "/content/nnUNet_results"

os.makedirs(os.environ["nnUNet_raw"], exist_ok=True)
os.makedirs(os.environ["nnUNet_preprocessed"], exist_ok=True)
os.makedirs(os.environ["nnUNet_results"], exist_ok=True)

In [32]:
# Pre-processing
!nnUNetv2_plan_and_preprocess -d 502 --verify_dataset_integrity

Fingerprint extraction...
Dataset502_KiTS23
Traceback (most recent call last):
  File "/usr/local/bin/nnUNetv2_plan_and_preprocess", line 8, in <module>
    sys.exit(plan_and_preprocess_entry())
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/nnUNet/nnunetv2/experiment_planning/plan_and_preprocess_entrypoints.py", line 180, in plan_and_preprocess_entry
    extract_fingerprints(args.d, args.fpe, args.npfp, args.verify_dataset_integrity, args.clean, args.verbose)
  File "/content/nnUNet/nnunetv2/experiment_planning/plan_and_preprocess_api.py", line 47, in extract_fingerprints
    extract_fingerprint_dataset(d, fingerprint_extractor_class, num_processes, check_dataset_integrity, clean,
  File "/content/nnUNet/nnunetv2/experiment_planning/plan_and_preprocess_api.py", line 30, in extract_fingerprint_dataset
    verify_dataset_integrity(join(nnUNet_raw, dataset_name), num_processes)
  File "/content/nnUNet/nnunetv2/experiment_planning/verify_dataset_integrity.py", line 191, in veri

In [ ]:
# training
!nnUNetv2_train -d 502 -c 3d_fullres -f 0 --max_num_epochs 30

In [ ]:
#visualization
